In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
import pandas as pd
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output, State
import plotly.graph_objs as go
from datetime import datetime,timedelta

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: UserWarning: 
The dash_core_components package is deprecated. Please replace
`import dash_core_components as dcc` with `from dash import dcc`
  """
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: UserWarning: 
The dash_html_components package is deprecated. Please replace
`import dash_html_components as html` with `from dash import html`
  


<span style="color: green; font-size:30px; font-weight:bold ">Mục đích chính của file "sentimentvisualizer" là:</span>  

Tạo giao diện trực quan hóa dữ liệu: File này thiết lập một ứng dụng Dash để hiển thị dữ liệu phân tích cảm xúc từ Twitter dưới dạng biểu đồ và bảng.

Kết nối với MongoDB: Ứng dụng này đọc dữ liệu từ cơ sở dữ liệu MongoDB, nơi lưu trữ các tweet và dự đoán cảm xúc.

Cập nhật dữ liệu theo thời gian thực: Ứng dụng có khả năng tự động cập nhật biểu đồ và bảng hiển thị dữ liệu mới nhất theo thời gian thực.

<span style="color: blue; font-size:20px; font-weight:bold ">Tạo SparkSession và cấu hình để tương tác với Kafka và MongoDB</span> 

In [2]:
#Spark Session creation configured to interact with MongoDB
spark = SparkSession.builder.appName("pyspark-notebook").\
config("spark.jars.packages","org.apache.spark:spark-sql-kafka-0-10_2.12:3.0.0,org.apache.spark:spark-avro_2.12:3.0.0,org.mongodb.spark:mongo-spark-connector_2.12:3.0.0").\
config("spark.mongodb.input.uri","mongodb://ubuntu_mongo_1:27017/twitter_db.tweets").\
config("spark.mongodb.output.uri","mongodb://ubuntu_mongo_1:27017/twitter_db.tweets").\
getOrCreate()

Ivy Default Cache set to: /root/.ivy2/cache
The jars for the packages stored in: /root/.ivy2/jars
:: loading settings :: url = jar:file:/usr/local/lib/python3.7/dist-packages/pyspark/jars/ivy-2.4.0.jar!/org/apache/ivy/core/settings/ivysettings.xml
org.apache.spark#spark-sql-kafka-0-10_2.12 added as a dependency
org.apache.spark#spark-avro_2.12 added as a dependency
org.mongodb.spark#mongo-spark-connector_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-af93bed0-965d-4cda-b999-0145abb98c14;1.0
	confs: [default]
	found org.apache.spark#spark-sql-kafka-0-10_2.12;3.0.0 in central
	found org.apache.spark#spark-token-provider-kafka-0-10_2.12;3.0.0 in central
	found org.apache.kafka#kafka-clients;2.4.1 in central
	found com.github.luben#zstd-jni;1.4.4-3 in central
	found org.lz4#lz4-java;1.7.1 in central
	found org.xerial.snappy#snappy-java;1.1.7.5 in central
	found org.slf4j#slf4j-api;1.7.30 in central
	found org.spark-project.spark#unused;1.0.0 in

SparkSession.builder: Tạo một phiên làm việc Spark mới với tên là "pyspark-notebook".

.config: Cấu hình các gói thư viện cần thiết:
        
    spark-sql-kafka-0-10: Để đọc dữ liệu từ Kafka.
    spark-avro: Để làm việc với định dạng Avro.
    mongo-spark-connector: Để kết nối và làm việc với MongoDB.
    
spark.mongodb.input.uri và spark.mongodb.output.uri: Địa chỉ kết nối đến cơ sở dữ liệu MongoDB để đọc và ghi dữ liệu.

<span style="color: blue; font-size:20px; font-weight:bold ">Tạo Ứng Dụng Dash</span> 

In [3]:
app = dash.Dash(__name__)

#Color assignment
colors = {
    'background': 'white',#'#0C0F0A',
    'text': '#FFFFFF'
}

def create_header(title):
    """Takes the input and Returns a html header

    Parameters
    ----------
    title : String
        Title of the Dashboard
        
    Returns
    ----------
        header: html header
    """
    
    header_style = {
        'background-color' : '#1B95E0',
        'padding' : '1.5rem',
        'color': 'white',
        'font-family': 'Verdana, Geneva, sans-serif'
    }
    header = html.Header(html.H1(children=title, style=header_style))
    return header
# create_header: Tạo một tiêu đề HTML với kiểu dáng và màu sắc cụ thể.


def generate_table(df, max_rows=10):
    """Takes pandas dataframe, optional max number of rows to display and returns html table

    Parameters
    ----------
    df : DataFrame
        Pandas dataframe
    max_rows: int
        Number of max rows to fit in a table
        
    Returns
    ----------
        table: html table
    """
    
    table = html.Table(className="responsive-table",
                      children=[
                          html.Thead(
                              html.Tr(
                                  children=[html.Th(col.title()) for col in df.columns.values]
                                  )
                              ),
                          html.Tbody(
                              [
                              html.Tr(
                                  children=[html.Td(data) for data in d]
                                  )
                               for d in df.values.tolist()])
                          ]
    )
    
    return table
# generate_table: Tạo bảng HTML từ DataFrame Pandas. Hiển thị tối đa max_rows hàng.


#Layout definition - contains a header, input box to get search term, a graph and a table
app.layout = html.Div(style={'backgroundColor': colors['background']}, children=
    [   
        html.Div([create_header('Live Dashboard - Twitter Sentiment Analysis')]),
        html.Div(["Serch Term: ", dcc.Input(id='sentiment_term', value='twitter', type='text',placeholder='Enter word to be searched'),
                  dcc.Graph(id='live-graph', animate=False)
                 ]
                 ,style={'width': '64%', 'display': 'inline-block'}
                ),
        html.Div([html.H2("Recent Tweets"), 
                  html.Div(id="recent-tweets-table")]
                 ,style={'width': '34%', 'display': 'inline-block'}
                ),
        #Intervals define the frequency in which the html element should be updated
        dcc.Interval(id='graph-update',interval=1*1000, n_intervals=0),
        dcc.Interval(id='recent-table-update',interval=10*1000, n_intervals=0)
    ]
)

# html.Div: Chứa các phần tử giao diện.
# create_header: Thêm tiêu đề cho dashboard.
# dcc.Input: Một ô nhập liệu để người dùng tìm kiếm từ khóa.
# dcc.Graph: Hiển thị biểu đồ cảm xúc.
# dcc.Interval: Cập nhật định kỳ cho biểu đồ và bảng dữ liệu.

<span style="color: blue; font-size:20px; font-weight:bold ">Callback để Cập Nhật Biểu Đồ</span> 

In [ ]:
#Call back for live graph
@app.callback(Output('live-graph', 'figure'),
              Input('graph-update', 'n_intervals'),
              Input('sentiment_term', 'value')
             )
# Hàm callback này cập nhật biểu đồ mỗi khi có dữ liệu mới từ MongoDB. 
# Nó lấy dữ liệu trong 200 giây qua và lọc theo từ khóa tìm kiếm.
def update_graph_scatter(n_intervals,sentiment_term):
    """Takes interval and search term as inputs and returs live-graph

    Parameters
    ----------
    n_intervals : int
        Frequency to update figure
    sentiment_term: int
        Search term to analyse the sentiment
        
    Returns
    ----------
        graph: html graph
        live-graph
    """
    try:
        #Read data from MongoDB for last 200 seconds
        time_diff = (datetime.utcnow() - timedelta(seconds=200)).strftime('%Y-%m-%d %H:%M:%S')
        df = spark.read.format("mongo").load().select("timestamp_ms","text","prediction").where("timestamp_ms>'"+time_diff+"' and lower(text) like lower('%"+sentiment_term+"%')").toPandas()
        df.sort_values('timestamp_ms', inplace=True)
        df.dropna(inplace=True)
        
        #Define X and Y axis values        
        X = df["timestamp_ms"]
        Y = df['prediction']#[-100:]
        
        #Scatter graph definition
        data = go.Scatter(
                x=X,
                y=Y,
                name='Scatter',
                mode= 'lines+markers'
                )

        return {'data': [data],'layout' : go.Layout(xaxis=dict(range=[X.min(),X.max()]),
                                                    yaxis=dict(range=[0,1]),
                                                    title='Twitter Sentiment {}'.format(sentiment_term)
                                                   )
               } 

    except Exception as e:
        #File to capture exceptions
        with open('errors.txt','a') as f:
            f.write(str(e))
            f.write('\n')

# Callback: Cập nhật biểu đồ khi có sự thay đổi:
# Đọc dữ liệu từ MongoDB trong vòng 200 giây qua Spark.
# Chọn các cột cần thiết và lọc theo từ khóa tìm kiếm.
# Vẽ biểu đồ phân tán (scatter plot) với các giá trị timestamp_ms và prediction.
# Xử lý lỗi và ghi vào file errors.txt.

#Call back for table to populate latest 10 tweets
@app.callback(Output('recent-tweets-table', 'children'),
              Input('recent-table-update', 'n_intervals'),
              Input('sentiment_term', 'value')
             )
# Hàm callback này cập nhật bảng hiển thị 5 tweet gần đây dựa trên từ khóa tìm kiếm.

def update_recent_tweets(n_intervals,sentiment_term):
    """Takes interval and search term as inputs and returs live-graph

    Parameters
    ----------
    n_intervals : int
        Frequency to update figure
    sentiment_term: int
        Search term to analyse the sentiment
        
    Returns
    ----------
        table: html graph
        table of latest 10 tweets
    """
    
    try:
        #Read data from MongoDB for last 200 seconds
        time_diff = (datetime.utcnow() - timedelta(seconds=200)).strftime('%Y-%m-%d %H:%M:%S')
        df = spark.read.format("mongo").load().select("timestamp_ms","text","prediction").where("timestamp_ms>'"+time_diff+"' and lower(text) like lower('%"+sentiment_term+"%')").limit(5).toPandas()
        df['sentiment'] = df['prediction']
        df['timestamp'] = df['timestamp_ms']
        df['tweet']     = df['text']

        df.drop(['timestamp_ms','text'],axis=1)

        df = df[['timestamp','tweet','sentiment']]

        return generate_table(df, max_rows=5)
    except Exception as e:
        #File to capture exceptions
        with open('table_errors.txt','a') as f:
            f.write(str(e))
            f.write('\n')

# Callback: Cập nhật bảng với 5 tweet gần đây:
# Đọc dữ liệu từ MongoDB trong vòng 200 giây qua Spark.
# Lọc dữ liệu theo từ khóa tìm kiếm.
# Tạo bảng HTML từ DataFrame và trả về kết quả.
# Xử lý lỗi và ghi vào file table_errors.txt.
if __name__ == '__main__':
    app.run_server(debug=False, use_reloader=False, port=8050,host= '0.0.0.0')

Dash is running on http://0.0.0.0:8050/

 * Serving Flask app '__main__' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on all addresses.
 * Running on http://172.18.0.7:8050/ (Press CTRL+C to quit)
